In [1]:
!cp -r /kaggle/input/anime-image-resolution-enhancement-psnr/* /kaggle/working/

# Install Dependencies

In [2]:
# !git clone https://github.com/xinntao/Real-ESRGAN.git
# or
#!git clone https://github.com/danhtran2mind/Real-ESRGAN.git
%cd Real-ESRGAN
!pip install --use-pep517 -q
!pip install basicsr==1.4.2 -q
!pip install facexlib -q
!pip install gfpgan -q
!pip install -r requirements.txt -q
!python setup.py develop --quiet
!pip install torch==2.0.1 torchvision==0.15.2 --extra-index-url https://download.pytorch.org/whl/cu118 -q
!pip install numpy==1.26.4 -q

/kaggle/working/Real-ESRGAN
ERROR: You must give at least one requirement to install (see "pip help install")
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 72.0 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 1.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 5.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 322.9 kB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 4.3 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.8 MB/s eta 0:00:00
/usr/local/lib/python3.11/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs ar

In [3]:
from pathlib import Path
from tqdm import tqdm
from tabulate import tabulate
import os
import numpy as np
from PIL import Image
import math
import subprocess
import torch
import shutil


In [ ]:
from huggingface_hub import snapshot_download, login
login("<your_huggingface_token>")

In [ ]:
# from huggingface_hub import HfApi
from huggingface_hub import snapshot_download
import os

# Define the dataset name and local directory
main_dir = "/kaggle/working/a"
up_dir = os.path.join(main_dir, "abc")

os.makedirs(main_dir, exist_ok=True)
os.makedirs(up_dir, exist_ok=True)

repo_id = "danhtran2mind/finetune-Real-ESRGAN-anime"
save_path = main_dir

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Download the dataset
snapshot_download(repo_id=repo_id, repo_type="dataset", local_dir=save_path)


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/2.46k [00:00<?, ?B/s]

file_names_bacth.json:   0%|          | 0.00/724k [00:00<?, ?B/s]

'/kaggle/working/a'

In [ ]:
# To temporary Model hub
from huggingface_hub import HfApi
from huggingface_hub import snapshot_download
# Initialize API
api = HfApi()

def upload_hf_dataset(save_dir="/kaggle/working/a/abc"):
    os.makedirs(save_dir, exist_ok=True)    
    # Upload the folder to the repository root
    api.upload_folder(
        folder_path=save_dir,  # Local folder path
        repo_id="danhtran2mind/finetune-Real-ESRGAN-anime",
        repo_type="dataset"
    )

In [8]:
def run_inference(device, model_name, input_path, output_dir, outscale, model_path):
    """Run Real-ESRGAN inference."""
    cmd = ["python", "inference_realesrgan.py", "-n", model_name, "-i", input_path, 
           "-o", output_dir, "--outscale", str(outscale), "--model_path", model_path]
    if device == "cpu":
        cmd.append("--fp32")
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

def calculate_psnr(img1_path, img2_path):
    """Calculate PSNR between two images."""
    try:
        img1, img2 = [np.array(Image.open(p).convert('RGB')) for p in [img1_path, img2_path]]
        if img1.shape != img2.shape:
            raise ValueError("Images must have same dimensions")
        mse = np.mean((img1 - img2) ** 2)
        return float('inf') if mse == 0 else 20 * math.log10(255.0 / math.sqrt(mse))
    except Exception as e:
        print(f"Error calculating PSNR for {img1_path} vs {img2_path}: {e}")
        return None

def get_image_paths(base_name, paths, subfolder):
    """Generate image file paths."""
    input_img = f"{subfolder}/{base_name}T1.png"
    raw_img = Path(paths['raw']) / f"{base_name}.jpg"
    if not raw_img.exists():
        raw_img = Path(paths['raw']) / f"{base_name}.png"
    return input_img, str(raw_img), f"{paths['original']}/{base_name}T1_out.png", f"{paths['finetune']}/{base_name}T1_out.png"

def create_subfolder(images, chunk_idx, chunk_size, paths):
    """Create a subfolder with a chunk of images copied from multiscale."""
    subfolder = Path(paths['inference']) / f"chunk_{chunk_idx}"
    subfolder.mkdir(parents=True, exist_ok=True)
    
    for img_name in images:
        src_img = Path(paths['multiscale']) / f"{img_name}T1.png"
        if src_img.exists():
            shutil.copy(src_img, subfolder / f"{img_name}T1.png")
    
    return str(subfolder)

def clear_inference_test_dir(inference_path):
    """Clear the inference test directory."""
    if os.path.exists(inference_path):
        shutil.rmtree(inference_path)
    os.makedirs(inference_path)
    
def save_to_json(original_psnr, finetuned_psnr, no_order,
                 or_dir="/kaggle/working/a/abc"):
    data = {"original_psnr": original_psnr,
            "finetuned_psnr": finetuned_psnr}
    
    pns_data_dir = os.path.join(or_dir, str(no_order))
    os.makedirs(pns_data_dir, exist_ok=True)
    pns_data_file = os.path.join(pns_data_dir, f"psnr_data_{no_order}.json")
    
    with open(pns_data_file, 'w') as f:
        json.dump(data, f, indent=4)


def process_images(device, base_names, paths, model_configs, chunk_size, no_order, completed_indexes):
    """Process images in chunks by creating subfolders and calculate PSNR values."""
    
    chunk_file_names = []  # To store file names for each chunk
    
    # Split base_names into chunks
    for chunk_idx in tqdm(range(0, len(base_names), chunk_size), desc="Processing chunks"):
        original_psnr, finetuned_psnr = [], []
        if chunk_idx in completed_indexes * chunk_size:
            print("Pass processed state")
            no_order += 1
            continue
        print(f"Do {chunk_idx}")
        chunk = base_names[chunk_idx:chunk_idx + chunk_size]
        
        # Store file names for this chunk
        chunk_file_names.append(chunk)
        
        # Create a subfolder for this chunk
        subfolder = create_subfolder(chunk, chunk_idx // chunk_size, chunk_size, paths)
        
        # Run inference on the subfolder
        for model_type, model_path in model_configs:
            run_inference(device, "RealESRGAN_x4plus", subfolder, paths[model_type], 2, model_path)
        
        # Process each image in the chunk
        for base_name in chunk:
            input_img, raw_img, original_img, finetuned_img = get_image_paths(base_name, paths, subfolder)
            
            if not os.path.exists(input_img):
                print(f"Input image not found: {input_img}")
                continue
            
            if all(os.path.exists(p) for p in [raw_img, original_img, finetuned_img]):
                for img, psnr_list, model_type in [(original_img, original_psnr, "Original"), 
                                                 (finetuned_img, finetuned_psnr, "Finetuned")]:
                    psnr = calculate_psnr(raw_img, img)
                    if psnr is not None:
                        psnr_list.append(psnr)
                        # print(f"{model_type} Model PSNR for {base_name}: {psnr:.4f}")
        
        save_to_json(original_psnr, finetuned_psnr, no_order)
        upload_hf_dataset()
        # Clear the subfolder and temp inference after processing
        clear_inference_test_dir(paths["inference"])
        clear_inference_test_dir(subfolder)
        
        no_order += 1
        
    return original_psnr, finetuned_psnr, chunk_file_names

In [9]:
# Configuration
MODEL_CONFIGS = [
    ("original", "./experiments/pretrained_models/RealESRGAN_x4plus.pth"),
    ("finetune", "./experiments/finetune_RealESRGAN_anime/models/net_g_latest.pth")
]
PATHS = {
    "raw": "/kaggle/input/anime-images-raw",
    "multiscale": "/kaggle/input/anime-images-multiscale",
    "original": "/content/inference_test/original",
    "finetune": "/content/inference_test/my_finetune",
    "inference": "/content/inference_test"
}
CHUNK_SIZE = 1000
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# # Get image files
# files = [f for f in os.listdir(PATHS["raw"]) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
# base_names = [os.path.splitext(f)[0] for f in (files[:total_images] if 'total_images' in globals() else files)]

# # Run PSNR extraction
# original_psnr, finetuned_psnr, chunk_file_names = process_images(device, base_names, PATHS, MODEL_CONFIGS, CHUNK_SIZE)

In [ ]:
import json

worker = 0
on_time = 7
completed_indexes = []
no_order = worker * on_time
psnr_data = f"/content/abc/psnr_data_{worker}.json"

with open('/kaggle/working/a/file_names_bacth.json', 'r') as f:
    data = json.load(f)

base_names = [name for i in range(on_time) for name in data[str(worker * on_time + i)]]

original_psnr, finetuned_psnr, _ = process_images(device, base_names, PATHS,
                                        MODEL_CONFIGS, CHUNK_SIZE, no_order, completed_indexes)

Processing chunks:   0%|          | 0/7 [00:00<?, ?it/s]

Do 0
